In [ ]:
import pandas as pd

data = pd.read_csv("data_input/trending.csv")
data.head()

In [3]:
data.columns

Index(['video_id', 'publish_time', 'channel_id', 'title', 'description',
       'thumbnail_url', 'thumbnail_width', 'thumbnail_height', 'channel_name',
       'tags', 'category_id', 'live_status', 'local_title',
       'local_description', 'duration', 'dimension', 'definition', 'caption',
       'license_status', 'allowed_region', 'blocked_region', 'view', 'like',
       'dislike', 'favorite', 'comment', 'trending_time'],
      dtype='object')

In [4]:
data = data[[ 'title', 'channel_name', 'category_id', 'view', 'like', 'dislike', 'comment', 'trending_time']]
data.head()

,title,channel_name,category_id,view,like,dislike,comment,trending_time
0,aespa 에스파 'Forever (약속)' MV,SMTOWN,10,7806846.0,857902.0,24078.0,94961.0,2021-02-07 05:46:51.832614+00:00
1,[FULL] Siapa di Balik Kudeta AHY? | Dua Sisi t...,Indonesia Lawyers Club,25,885038.0,6765.0,1263.0,6365.0,2021-02-07 05:46:51.832649+00:00
2,CABRIOLET CHALLENGE: TANTANGAN MENGGODA (7/12),Motomobi,2,889708.0,47895.0,532.0,8785.0,2021-02-07 05:46:51.832664+00:00
3,With Windah Basudara & Hans,yb,20,1006854.0,91973.0,3967.0,12957.0,2021-02-07 05:46:51.832678+00:00
4,🤯 LATE COMEBACK DRAMA! | HIGHLIGHTS | Granada ...,FC Barcelona,17,6275035.0,218131.0,4289.0,12799.0,2021-02-07 05:46:51.832730+00:00


In [14]:
import json

with open("data_input/category.json", "rb") as f:
    categories = json.load(f)

In [15]:
categories = pd.json_normalize(categories, record_path="items")
categories_dict = {idx:title for idx, title in zip(categories['id'], categories['snippet.title'])}

In [17]:
data['category'] = [categories_dict[str(x)] if str(x) in list(categories_dict.keys()) else "Unknown" for x in data['category_id']]

In [18]:
data

,title,channel_name,category_id,view,like,dislike,comment,trending_time,category
0,aespa 에스파 'Forever (약속)' MV,SMTOWN,10,7806846.0,857902.0,24078.0,94961.0,2021-02-07 05:46:51.832614+00:00,Music
1,[FULL] Siapa di Balik Kudeta AHY? | Dua Sisi t...,Indonesia Lawyers Club,25,885038.0,6765.0,1263.0,6365.0,2021-02-07 05:46:51.832649+00:00,News & Politics
2,CABRIOLET CHALLENGE: TANTANGAN MENGGODA (7/12),Motomobi,2,889708.0,47895.0,532.0,8785.0,2021-02-07 05:46:51.832664+00:00,Autos & Vehicles
3,With Windah Basudara & Hans,yb,20,1006854.0,91973.0,3967.0,12957.0,2021-02-07 05:46:51.832678+00:00,Gaming
4,🤯 LATE COMEBACK DRAMA! | HIGHLIGHTS | Granada ...,FC Barcelona,17,6275035.0,218131.0,4289.0,12799.0,2021-02-07 05:46:51.832730+00:00,Sports
...,...,...,...,...,...,...,...,...,...
172342,IBT 167: A IKY BONGKAR GEBETAN BARU SULE?! TEB...,SULE PRODUCTIONS,24,841847.0,27057.0,NaN,1251.0,2023-09-01 09:16:49.916190+00:00,Entertainment
172343,Sangkolan 1 | short movie madura ( SUB INDONES...,Akeloy Production,24,288609.0,8815.0,NaN,618.0,2023-09-01 09:16:49.916196+00:00,Entertainment
172344,BONGKAR RUMAH MASA KECILKU DI DESA WAJOK HILIR...,YENNY DI CHINA,22,1056069.0,32632.0,NaN,6445.0,2023-09-01 09:16:49.916201+00:00,People & Blogs
172345,Apakah Ini KODE KERAS Zidan ‼️ Merayu Mbak Yay...,musisi jogja project,10,1775895.0,17359.0,NaN,868.0,2023-09-01 09:16:49.916206+00:00,Music


In [19]:
data['category'].value_counts()

category
Entertainment           48625
Music                   39986
People & Blogs          21094
Gaming                  16486
Sports                  14790
News & Politics          7679
Film & Animation         6122
Comedy                   4971
Autos & Vehicles         3929
Howto & Style            2330
Science & Technology     2204
Pets & Animals           1344
Education                1278
Travel & Events          1023
Unknown                   486
Name: count, dtype: int64

In [22]:
data.isnull().sum()

title            0
channel_name     0
category_id      0
view             0
like             0
dislike          0
comment          0
trending_time    0
category         0
dtype: int64

In [29]:
data = data.fillna(0)

In [32]:
try:
    data['trending_time'] = pd.to_datetime(data['trending_time'], format='ISO8601', errors='coerce')
except ValueError:
    print("Format waktu tidak konsisten, mencoba cara lain...")

# Periksa nilai yang gagal dikonversi (jika ada)
if data['trending_time'].isna().sum() > 0:
    print("Beberapa data tidak valid dan diatur sebagai NaT.")

# Ekstrak tanggal saja dari waktu
data['trending_date'] = data['trending_time'].dt.date

# Hapus kolom lama yang sudah tidak diperlukan
data = data.drop(['trending_time'], axis=1)

# Tampilkan hasil
print(data.head())

                                               title            channel_name  \
0                        aespa 에스파 'Forever (약속)' MV                  SMTOWN   
1  [FULL] Siapa di Balik Kudeta AHY? | Dua Sisi t...  Indonesia Lawyers Club   
2     CABRIOLET CHALLENGE: TANTANGAN MENGGODA (7/12)                Motomobi   
3                        With Windah Basudara & Hans                      yb   
4  🤯 LATE COMEBACK DRAMA! | HIGHLIGHTS | Granada ...            FC Barcelona   

   category_id       view      like  dislike  comment          category  \
0           10  7806846.0  857902.0  24078.0  94961.0             Music   
1           25   885038.0    6765.0   1263.0   6365.0   News & Politics   
2            2   889708.0   47895.0    532.0   8785.0  Autos & Vehicles   
3           20  1006854.0   91973.0   3967.0  12957.0            Gaming   
4           17  6275035.0  218131.0   4289.0  12799.0            Sports   

  trending_date  
0    2021-02-07  
1    2021-02-07  
2    2021-02-0

In [33]:
data

,title,channel_name,category_id,view,like,dislike,comment,category,trending_date
0,aespa 에스파 'Forever (약속)' MV,SMTOWN,10,7806846.0,857902.0,24078.0,94961.0,Music,2021-02-07
1,[FULL] Siapa di Balik Kudeta AHY? | Dua Sisi t...,Indonesia Lawyers Club,25,885038.0,6765.0,1263.0,6365.0,News & Politics,2021-02-07
2,CABRIOLET CHALLENGE: TANTANGAN MENGGODA (7/12),Motomobi,2,889708.0,47895.0,532.0,8785.0,Autos & Vehicles,2021-02-07
3,With Windah Basudara & Hans,yb,20,1006854.0,91973.0,3967.0,12957.0,Gaming,2021-02-07
4,🤯 LATE COMEBACK DRAMA! | HIGHLIGHTS | Granada ...,FC Barcelona,17,6275035.0,218131.0,4289.0,12799.0,Sports,2021-02-07
...,...,...,...,...,...,...,...,...,...
172342,IBT 167: A IKY BONGKAR GEBETAN BARU SULE?! TEB...,SULE PRODUCTIONS,24,841847.0,27057.0,0.0,1251.0,Entertainment,2023-09-01
172343,Sangkolan 1 | short movie madura ( SUB INDONES...,Akeloy Production,24,288609.0,8815.0,0.0,618.0,Entertainment,2023-09-01
172344,BONGKAR RUMAH MASA KECILKU DI DESA WAJOK HILIR...,YENNY DI CHINA,22,1056069.0,32632.0,0.0,6445.0,People & Blogs,2023-09-01
172345,Apakah Ini KODE KERAS Zidan ‼️ Merayu Mbak Yay...,musisi jogja project,10,1775895.0,17359.0,0.0,868.0,Music,2023-09-01


In [34]:
data[['title', 'channel_name']].value_counts().reset_index(name='counts')

,title,channel_name,counts
0,Top Lagu Pop Indonesia Terbaru 2021 Hits Pilih...,nayan barari,78
1,Official Trailer | She-Hulk: Attorney at Law |...,Marvel Entertainment,38
2,Toleransi antar tetangga di Korea🇰🇷.. 🤦🏻‍♂️,Korea Reomit,37
3,Yeni Inka - PECAH SERIBU (Official MV) Hanya D...,Aneka Safari Records,37
4,Yura Yunita - Tutur Batin (Official Music Video),YURA YUNITA,37
...,...,...,...
24753,NGINEP DI HOTEL NO.1 JEPANG! ADA WATERPARK DAN...,Nihongo Mantappu,1
24754,NGINEP DI HOTEL 18JT/MALAM DI DUBAI!,Jessica Jane,1
24755,"NGERJAIN ADIK BARENG PACAR , KASIAN TAPI LUCU ...",Tasya Revina,1
24756,NGASIH UANG 14 JUTA CASH + UANG DARI JUAL MOTO...,Bro Omen,1


In [35]:
used_data = data.drop_duplicates(['title', 'channel_name'], keep='first')

In [36]:
used_data = used_data.reset_index(drop=True)

In [38]:
used_data = used_data.drop(['category_id'], axis=1)
used_data.to_pickle('data_input/used_data.pickle')